In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

tf.version.VERSION

'2.4.0'

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

y_train = y_train[:1000]
y_test = y_test[:1000]
X_train = X_train[:1000].reshape(-1, 28*28) / 255.
X_test = X_test[:1000].reshape(-1, 28*28) / 255.

In [3]:
def create_mod():
    mod = Sequential([Dense(512, activation='relu', input_shape=(784,)),
                      Dropout(0.2),
                      Dense(10)])
    mod.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return mod

In [4]:
mod = create_mod()
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training

In [5]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1)
mod.fit(X_train,
        y_train,
        epochs=10,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 20ms/step - loss: 1.6129 - sparse_categorical_accuracy: 0.4945 - val_loss: 0.6859 - val_sparse_categorical_accuracy: 0.8000

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4047 - sparse_categorical_accuracy: 0.8987 - val_loss: 0.5337 - val_sparse_categorical_accuracy: 0.8330

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2960 - sparse_categorical_accuracy: 0.9308 - val_loss: 0.4880 - val_sparse_categorical_accuracy: 0.8410

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1911 - sparse_categorical_accuracy: 0.9444 - val_loss: 0.4758 - val_sparse_categorical_accuracy: 0.8510

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1563 - sparse_cat

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [7]:
mod = create_mod()
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Untrained: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 2.3816 - sparse_categorical_accuracy: 0.0990
Untrained: Acc:  9.90


In [8]:
mod.load_weights(checkpoint_path)
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Loaded weights: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 0.4343 - sparse_categorical_accuracy: 0.8630
Loaded weights: Acc: 86.30


### Checkpoint Callback Options

In [9]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

BATCH = 32

In [10]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5 * BATCH)

mod = create_mod()
mod.save_weights(checkpoint_path.format(epoch=0))
mod.fit(X_train,
        y_train,
        epochs=50,
        batch_size=BATCH,
        callbacks=[cp_callback],
        validation_data=(X_test, y_test),
        verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
mod = create_mod()
mod.load_weights(latest)
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Restored mod: Acc: {100 * acc:5.2f}%')

32/32 - 0s - loss: 0.4772 - sparse_categorical_accuracy: 0.8820
Restored mod: Acc: 88.20%


### Manually save weights

In [18]:
mod.save_weights('./checkpoints/my_checkpoint')
mod = create_mod()
mod.load_weights('./checkpoints/my_checkpoint')
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Restored mod: Acc: {100 * acc:5.2f}%')

32/32 - 0s - loss: 0.4772 - sparse_categorical_accuracy: 0.8820
Restored mod: Acc: 88.20%
